### 문제 1-1 : 기본 체인 만들기 - AI 요리사

In [9]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI 

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [10]:
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x00000218E0087110> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000218E0087380> root_client=<openai.OpenAI object at 0x00000218E0086510> root_async_client=<openai.AsyncOpenAI object at 0x00000218E00855B0> model_name='meta-llama/llama-4-scout-17b-16e-instruct' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [16]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""
당신은 요리 전문가입니다. 사용자가 재료를 입력하면 그 재료로 만들 수 있는 요리를 추천해주세요.
재료: {ingredients}


출력 형식:
입력: "{ingredients}"  
출력: {ingredients}로 만들 수 있는 요리를 추천드립니다!
                                      
추천 요리: [요리 이름]
재료: [입력된 재료]
조리법: 
1. ...
2. ...
""")

prompt


PromptTemplate(input_variables=['ingredients'], input_types={}, partial_variables={}, template='\n당신은 요리 전문가입니다. 사용자가 재료를 입력하면 그 재료로 만들 수 있는 요리를 추천해주세요.\n재료: {ingredients}\n\n\n출력 형식:\n입력: "{ingredients}"  \n출력: {ingredients}로 만들 수 있는 요리를 추천드립니다!\n\n추천 요리: [요리 이름]\n재료: [입력된 재료]\n조리법: \n1. ...\n2. ...\n')

In [17]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable

output_parser = StrOutputParser()

chain: Runnable = prompt | llm | output_parser

print(type(chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [18]:
try:
    result = chain.invoke({"ingredients": "계란, 밥, 김치"})
    print(type(result))
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'str'>
입력: "계란, 밥, 김치"  
출력: 계란, 밥, 김치로 만들 수 있는 요리를 추천드립니다!

추천 요리: 김치볶음밥
재료: 계란, 밥, 김치
조리법: 
1. 김치를 잘게 썰고, 계란은 풀어서 준비합니다.
2. 팬을 달군 후 썰어 놓은 김치를 넣고 볶습니다.
3. 김치가 볶아지면 풀어 놓은 계란을 넣고 잘 섞어줍니다.
4. 밥을 넣고 모든 재료가 잘 섞이도록 볶습니다.
5. 간을 맞추고 그릇에 담아내면 완성입니다.


### 문제 1-2 : 2단계 체인 만들기 - 영화 추천 시스템

In [45]:

from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 1단계 출력: 추천 영화 제목과 간단한 설명
system_msg1 = SystemMessagePromptTemplate.from_template(
    "당신은 영화 추천 전문가입니다. 사용자에게 장르에 맞는 영화 한 편을 추천해 주세요. "
    "형식은 반드시 다음과 같이 해 주세요: 영화 제목"
)
human_msg1 = HumanMessagePromptTemplate.from_template(
    "제가 좋아하는 영화 장르는 {genre}입니다."
)
recommend_prompt = ChatPromptTemplate.from_messages([system_msg1, human_msg1])

# 2단계 출력: 영화 줄거리 3줄 요약
system_msg2 = SystemMessagePromptTemplate.from_template(
    "당신은 영화 줄거리 요약 전문가입니다. 영화 제목을 바탕으로 3줄로 요약해 주세요.\n"
    "형식은 다음과 같이 해 주세요:\n"
    "1. ...\n2. ...\n3. ..."
)
human_msg2 = HumanMessagePromptTemplate.from_template(
    "영화 제목: {movie}"
)
summary_prompt = ChatPromptTemplate.from_messages([system_msg2, human_msg2])

llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

recommend_chain: Runnable = recommend_prompt | llm
summary_chain: Runnable = summary_prompt | llm

genre_input = {"genre": "액션"}

recommend_result = recommend_chain.invoke(genre_input)
movie_title = recommend_result.content.strip()

summary_result = summary_chain.invoke({"movie": movie_title})

print(f'입력: "액션"')
print(f'1단계 결과: "{movie_title}" 추천')
print("2단계 결과:")
print(f"영화: {movie_title}")
print("줄거리:")
print(summary_result.content.strip())

입력: "액션"
1단계 결과: "쥬만지: 새로운 세계" 추천
2단계 결과:
영화: 쥬만지: 새로운 세계
줄거리:
1. 비디오 게임 속으로 빨려 들어간 네 명의 고등학생들은 게임 속 세계인 쥬만지에서 살아남기 위해 힘겨운 싸움을 시작합니다.
2. 게임 속 세계에서 각자의 아바타로 변한 이들은 게임의 목적이 '쥬만지'를 탈출하는 것임을 알고, 각자의 목표를 달성하기 위해 협력합니다.
3. 게임 속 세계에서의 모험을 통해 성장하고 우정을 쌓은 네 명의 주인공들은 결국 게임의 비밀을 풀고 현실 세계로 돌아오게 됩니다.


### 문제 1-3 : FewShotPromptTemplate과 시스템 메시지 활용

In [ ]:
from langchain_core.prompts import (
    FewShotChatMessagePromptTemplate, 
    ChatPromptTemplate
)
from langchain_openai import ChatOpenAI

examples = [
    {
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    {
        "news": "테슬라는 자율주행 기술의 상용화를 위한 새로운 소프트웨어 업데이트를 발표했다. 이번 업데이트는 운전자의 개입을 최소화하고 차량의 주행 안정성을 높이는 데 중점을 두고 있다.",
        "keywords": "테슬라, 자율주행, 소프트웨어"
    }
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 뉴스 키워드 추출 전문가입니다. 핵심 키워드 3개를 추출하세요."),
    few_shot_prompt,
    ("human", "{input}")
])

llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.3
)

test_news = {
    "input": "제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."
}

messages = final_prompt.format_messages(**test_news)
response = llm.invoke(messages)

print(response.content.strip())

키워드: 제미나이, AI, 구글
